In [ ]:
import xarray as xr
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import numpy as np
import xesmf as xe
from pathlib import Path

In [ ]:
def abrir_modelos(variable, ruta_base=None):
    """
    Abrir todos los archivos .nc en la subcarpeta correspondiente a la variable dentro de 'modelos'.

    Returns:
      list[xarray.Dataset]: lista con los datasets abiertos.

    Raises:
      FileNotFoundError: si la carpeta no existe o no contiene archivos .nc.
    """
    try:
        base = Path(__file__).parent  # disponible si se ejecuta como script
    except NameError:
        base = Path.cwd()
    # Resolver ruta_base: por defecto 'modelos' relativa a este archivo,
    # pero permite pasar una ruta absoluta o relativa si se desea.
    if ruta_base:
        modelos_root = Path(ruta_base)
        if not modelos_root.is_absolute():
            modelos_root = base / modelos_root
    else:
        modelos_root = base / 'modelos'

    ruta_variable = modelos_root / variable
    if not ruta_variable.exists() or not ruta_variable.is_dir():
        raise FileNotFoundError(f"No existe la carpeta de la variable: {ruta_variable}")

    nc_files = sorted(ruta_variable.glob('*.nc'))
    if not nc_files:
        raise FileNotFoundError(f"No se encontraron archivos .nc en {ruta_variable}")

    modelos = [xr.open_dataset(str(archivo)) for archivo in nc_files]

    print(f"Se abrieron {len(modelos)} modelos para la variable '{variable}'.")
    return modelos

In [ ]:
modelos_clt = abrir_modelos('clt')
print(modelos_clt)

Se abrieron 7 modelos para la variable 'clt'.
[<xarray.Dataset> Size: 219MB
Dimensions:    (time: 1980, bnds: 2, lat: 144, lon: 192)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * lat        (lat) float64 1kB -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
  * lon        (lon) float64 2kB 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 32kB ...
    lat_bnds   (lat, bnds) float64 2kB ...
    lon_bnds   (lon, bnds) float64 3kB ...
    clt        (time, lat, lon) float32 219MB ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    creation_date:          2019-11-08T10:46:33Z
    ...                     ...
    variable_id:            clt
    variant_label:          r1i1p1f1
    version:                

c:\Users\123\anaconda3\envs\clima\Lib\site-packages\xarray\conventions.py:204: SerializationWarning: variable 'clt' has multiple fill values {np.float32(1e+20), np.float64(1e+20)} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


In [ ]:
print(modelos_clt[1]['clt'])

<xarray.DataArray 'clt' (time: 1980, lat: 145, lon: 192)> Size: 220MB
[55123200 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T12...
  * lat      (lat) float64 1kB -90.0 -88.75 -87.5 -86.25 ... 87.5 88.75 90.0
  * lon      (lon) float64 2kB 0.0 1.875 3.75 5.625 ... 352.5 354.4 356.2 358.1
Attributes:
    standard_name:   cloud_area_fraction
    long_name:       Total Cloud Cover Percentage
    comment:         Total cloud area fraction (reported as a percentage) for...
    units:           %
    original_units:  1
    history:         2019-11-15T07:21:23Z altered by CMOR: Converted units fr...
    cell_methods:    area: time: mean
    cell_measures:   area: areacella


In [ ]:
def info_grids(modelos):
    """
    Dada una lista de modelos (xarray.Dataset),
    devuelve un resumen con el tamaño y nombre de las coordenadas de cada grid.
    """
    info = []
    for i, ds in enumerate(modelos, start=1):
        # Buscar posibles nombres de coordenadas de lat/lon
        lat_name = next((c for c in ds.coords if 'lat' in c.lower()), None)
        lon_name = next((c for c in ds.coords if 'lon' in c.lower()), None)
        
        if lat_name and lon_name:
            lat_size = ds[lat_name].size
            lon_size = ds[lon_name].size
            info.append({
                'modelo': i,
                'lat_name': lat_name,
                'lon_name': lon_name,
                'lat_size': lat_size,
                'lon_size': lon_size
            })
        else:
            info.append({
                'modelo': i,
                'lat_name': lat_name,
                'lon_name': lon_name,
                'error': 'No se encontraron coordenadas lat/lon'
            })
    
    return info


In [ ]:
grids_info = info_grids(modelos_clt)
for g in grids_info:
    print(g)

{'modelo': 1, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 144, 'lon_size': 192}
{'modelo': 2, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 145, 'lon_size': 192}
{'modelo': 3, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 4, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 5, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}
{'modelo': 6, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 160, 'lon_size': 320}
{'modelo': 7, 'lat_name': 'lat', 'lon_name': 'lon', 'lat_size': 192, 'lon_size': 288}


In [ ]:
def regrid_all(modelos, var_name, ref_index):
    """
    Regridea todas las variables 'var_name' de los modelos (en una lista)
    al grid del modelo de referencia indicado por ref_index.

    Parámetros
    ----------
    modelos : list[xarray.Dataset | xarray.DataArray]
        Lista de modelos.
    var_name : str
        Nombre de la variable a regridear (ej: 'clt')
    ref_index : int
        Índice del modelo de referencia en la lista (ej: 2 para el tercero)
    
    Retorna
    -------
    list[xarray.DataArray]
        Lista de variables regrideadas al mismo grid.
    """
    regridded = []
    ref_model = modelos[ref_index]
    ref_grid = ref_model[var_name] if hasattr(ref_model, 'data_vars') else ref_model

    for i, model in enumerate(modelos):
        print(f"Procesando modelo {i}...")
        data = model[var_name] if hasattr(model, 'data_vars') else model
        
        if i == ref_index:
            regridded.append(data)
        else:
            regridder = xe.Regridder(
                data,
                ref_grid,
                method='bilinear',
                extrap_method='nearest_s2d',
                reuse_weights=False
            )
            regridded.append(regridder(data))
    
    return regridded

In [ ]:
clt_regridded = regrid_all(modelos_clt, 'clt', 3)


Procesando modelo 0...
Procesando modelo 1...
Procesando modelo 2...
Procesando modelo 3...
Procesando modelo 4...
Procesando modelo 5...
Procesando modelo 6...


In [ ]:
clt_regridded[0]

<xarray.DataArray (time: 1980, lat: 192, lon: 288)> Size: 438MB
array([[[ 92.08984 ,  92.08984 ,  92.08984 , ...,  92.08984 ,
          92.08984 ,  92.08984 ],
        [ 90.16113 ,  90.71877 ,  90.797615, ...,  90.343506,
          90.43462 ,  90.31896 ],
        [ 92.13867 ,  92.122665,  91.54311 , ...,  91.8848  ,
          91.95943 ,  92.0892  ],
        ...,
        [ 98.376465,  98.3929  ,  98.25576 , ...,  98.56112 ,
          98.52989 ,  98.40789 ],
        [ 97.58301 ,  97.78891 ,  97.75681 , ...,  97.98361 ,
          98.0244  ,  97.96001 ],
        [ 95.8313  ,  95.8313  ,  95.8313  , ...,  95.8313  ,
          95.8313  ,  95.8313  ]],

       [[ 93.85986 ,  93.85986 ,  93.85986 , ...,  93.85986 ,
          93.85986 ,  93.85986 ],
        [ 87.457275,  84.21403 ,  84.51717 , ...,  84.144615,
          84.42508 ,  84.43877 ],
        [ 73.04077 ,  74.3455  ,  74.27838 , ...,  73.25699 ,
          73.12719 ,  73.0439  ],
...
        [ 97.58911 ,  97.76902 ,  97.83558 , ...,  97.56226 ,
          97.62424 ,  97.59217 ],
        [ 98.12622 ,  98.05859 ,  98.16012 , ...,  97.98435 ,
          97.97282 ,  98.12846 ],
        [ 97.86377 ,  97.86377 ,  97.86377 , ...,  97.86377 ,
          97.86377 ,  97.86377 ]],

       [[ 85.2478  ,  85.2478  ,  85.2478  , ...,  85.2478  ,
          85.2478  ,  85.2478  ],
        [ 85.30884 ,  85.87049 ,  85.46315 , ...,  85.73601 ,
          85.54579 ,  85.6962  ],
        [ 87.30469 ,  87.77076 ,  87.28404 , ...,  88.078896,
          87.58661 ,  87.33572 ],
        ...,
        [ 93.65845 ,  93.17506 ,  93.61942 , ...,  93.464645,
          93.61676 ,  93.6393  ],
        [ 97.38159 ,  96.30644 ,  96.3837  , ...,  96.53952 ,
          96.64447 ,  96.3866  ],
        [ 97.19238 ,  97.19238 ,  97.19238 , ...,  97.19238 ,
          97.19238 ,  97.19238 ]]], shape=(1980, 192, 288), dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T12...
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    regrid_method:  bilinear

In [ ]:
def calc_monthly_means(modelos, var_name=None, start='1950-01-01', end='1980-12-31'):
    """
    Calcula la media mensual (1950–1980 por defecto) para una lista de modelos.
    
    Parámetros
    ----------
    modelos : list[xarray.Dataset | xarray.DataArray]
        Lista de modelos o variables.
    var_name : str | None
        Nombre de la variable si los elementos son Datasets (ej: 'clt').
        Si ya son DataArray, déjalo en None.
    start, end : str
        Fechas para el rango temporal (slice).
    
    Retorna
    -------
    list[xarray.DataArray]
        Lista con las medias mensuales de cada modelo.
    """
    monthly_means = []
    
    for i, model in enumerate(modelos):
        print(f"Calculando media mensual para modelo {i+1}...")
        
        # Extrae la variable si es Dataset
        data = model[var_name] if (var_name and hasattr(model, 'data_vars')) else model
        
        # Selección temporal y promedio mensual
        mean_month = (
            data
            .sel(time=slice(start, end))
            .groupby('time.month')
            .mean('time')
        )
        
        monthly_means.append(mean_month)
    
    return monthly_means


In [ ]:
clt_monthly_means = calc_monthly_means(clt_regridded, start='1950-01-01', end='1980-12-31')


Calculando media mensual para modelo 1...
Calculando media mensual para modelo 2...
Calculando media mensual para modelo 3...
Calculando media mensual para modelo 4...
Calculando media mensual para modelo 5...
Calculando media mensual para modelo 6...
Calculando media mensual para modelo 7...


In [ ]:
clt_monthly_means[2]

<xarray.DataArray (month: 12, lat: 192, lon: 288)> Size: 3MB
array([[[73.41167 , 73.41166 , 73.41165 , ..., 73.411644, 73.41167 ,
         73.411644],
        [69.170006, 69.1553  , 69.15957 , ..., 69.20637 , 69.20936 ,
         69.21547 ],
        [66.79935 , 66.723114, 66.71926 , ..., 66.87967 , 66.9004  ,
         66.866455],
        ...,
        [93.5121  , 93.52318 , 93.551926, ..., 93.57602 , 93.57219 ,
         93.52119 ],
        [93.356575, 93.385086, 93.41131 , ..., 93.355125, 93.3389  ,
         93.35705 ],
        [93.52743 , 93.527504, 93.52758 , ..., 93.528244, 93.52733 ,
         93.52734 ]],

       [[80.9864  , 80.9864  , 80.9864  , ..., 80.98639 , 80.9864  ,
         80.98639 ],
        [75.03262 , 75.01598 , 75.00732 , ..., 75.047195, 75.0491  ,
         75.053185],
        [72.57562 , 72.5519  , 72.52565 , ..., 72.58969 , 72.578285,
         72.57907 ],
...
        [94.166794, 94.14551 , 94.1301  , ..., 94.147606, 94.1575  ,
         94.15079 ],
        [94.31478 , 94.30713 , 94.336685, ..., 94.330475, 94.33915 ,
         94.33706 ],
        [94.25184 , 94.25231 , 94.25243 , ..., 94.25267 , 94.25263 ,
         94.25185 ]],

       [[70.65097 , 70.65097 , 70.65097 , ..., 70.651024, 70.65099 ,
         70.650986],
        [66.63268 , 66.61921 , 66.629715, ..., 66.67559 , 66.66746 ,
         66.66055 ],
        [64.30743 , 64.29715 , 64.29394 , ..., 64.35515 , 64.326805,
         64.34901 ],
        ...,
        [93.38011 , 93.42774 , 93.436455, ..., 93.23777 , 93.261696,
         93.295105],
        [93.557846, 93.59039 , 93.626076, ..., 93.44132 , 93.45776 ,
         93.50969 ],
        [93.51122 , 93.51136 , 93.51091 , ..., 93.509125, 93.5102  ,
         93.51023 ]]], shape=(12, 192, 288), dtype=float32)
Coordinates:
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Attributes:
    regrid_method:  bilinear

In [ ]:

def ensemble_mean(models_monthly):
    """
    Calcula la media del ensemble (promedio entre modelos).
    
    Parámetros
    ----------
    models_monthly : list[xarray.DataArray]
        Lista de DataArrays con las medias mensuales de cada modelo.
    
    Retorna
    -------
    xarray.DataArray
        DataArray con el promedio del ensemble.
    """
    # Concatenamos a lo largo de un eje ficticio 'model'
    combined = xr.concat(models_monthly, dim='model')
    return combined.mean(dim='model')

In [ ]:
clt_ensemble_mean = ensemble_mean(clt_monthly_means)
clt_ensemble_mean

<xarray.DataArray (month: 12, lat: 192, lon: 288)> Size: 3MB
array([[[62.91795 , 62.917793, 62.91801 , ..., 62.917908, 62.91803 ,
         62.917786],
        [59.944912, 60.03253 , 60.011806, ..., 60.13266 , 60.085583,
         59.93748 ],
        [59.873116, 59.89207 , 59.765114, ..., 59.728855, 59.6141  ,
         59.883495],
        ...,
        [75.401505, 75.40331 , 75.42194 , ..., 75.37994 , 75.38841 ,
         75.36688 ],
        [75.422295, 75.39049 , 75.38927 , ..., 75.36261 , 75.37266 ,
         75.36916 ],
        [75.473526, 75.47353 , 75.47355 , ..., 75.47368 , 75.47353 ,
         75.47351 ]],

       [[69.86372 , 69.86372 , 69.86372 , ..., 69.86372 , 69.86372 ,
         69.86372 ],
        [67.004524, 66.87387 , 66.85417 , ..., 66.98646 , 66.95876 ,
         66.84709 ],
        [64.93791 , 64.97557 , 64.98099 , ..., 64.90978 , 64.91988 ,
         64.926186],
...
        [78.27837 , 78.27351 , 78.28794 , ..., 78.27117 , 78.28562 ,
         78.2384  ],
        [78.5079  , 78.468506, 78.48009 , ..., 78.453575, 78.455   ,
         78.45441 ],
        [78.39345 , 78.393875, 78.39411 , ..., 78.392815, 78.39304 ,
         78.39332 ]],

       [[58.281746, 58.28172 , 58.281746, ..., 58.28184 , 58.281757,
         58.28171 ],
        [55.26405 , 55.20898 , 55.18404 , ..., 55.33988 , 55.287884,
         55.016144],
        [54.81396 , 54.782948, 54.682526, ..., 54.760162, 54.643948,
         54.812107],
        ...,
        [75.936134, 75.97151 , 75.977486, ..., 75.9036  , 75.90581 ,
         75.92773 ],
        [76.23181 , 76.20973 , 76.214096, ..., 76.14771 , 76.16564 ,
         76.174736],
        [76.21765 , 76.21767 , 76.21763 , ..., 76.21737 , 76.217514,
         76.217514]]], shape=(12, 192, 288), dtype=float32)
Coordinates:
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8